In [247]:
import pandas as pd
import joblib
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV

In [248]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [249]:
O_E = OrdinalEncoder()
train[['id','Target']] = O_E.fit_transform(train[['id','Target']])
train.head(2)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2.0
1,1.0,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0.0


In [ ]:
duplicates = train[train.duplicated()]
print(duplicates)

In [250]:
train.corr()['Target'].sort_values(ascending=False)

Target                                            1.000000
Curricular units 2nd sem (approved)               0.781452
Curricular units 1st sem (approved)               0.725490
Curricular units 2nd sem (grade)                  0.719036
Curricular units 1st sem (grade)                  0.661355
Tuition fees up to date                           0.415691
Scholarship holder                                0.394124
Curricular units 2nd sem (enrolled)               0.289165
Curricular units 1st sem (enrolled)               0.263657
Curricular units 2nd sem (evaluations)            0.214951
Admission grade                                   0.172880
Course                                            0.154208
Curricular units 1st sem (evaluations)            0.152398
Displaced                                         0.150066
Previous qualification (grade)                    0.138119
Application order                                 0.128394
Daytime/evening attendance                        0.1244

Drop clounms that has no correlation with indipendent variable
1 Educational special needs  
2 International
3 Nacionality
4 Unemployment rate
5 Curricular units 1st sem (credited) 
6 Curricular units 2nd sem (credited)
7 Mother's qualification
8 Previous qualification
9 Father's qualification
10 Father's occupation  
11 Inflation rate 

In [ ]:
train = train.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
test = test.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
outliers = ((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR)))


In [ ]:
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]


In [ ]:
print(train.describe())

In [ ]:
param_grid_decision_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6, 8]
}

In [ ]:
model = DecisionTreeClassifier(random_state=42)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid_decision_tree, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

In [ ]:
features= train.drop(['id', 'Target'],axis=1)
target=train['Target']# store the labels .to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
X_train_pred = model.predict(X_train)

In [ ]:
trainingAccuracy = accuracy_score(y_train, X_train_pred)
print(f"Training Accuracy of Decision Tree: {trainingAccuracy:.2f}")

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
testset = test.drop('id' , axis =1)

In [ ]:
pred = best_model.predict(testset)

In [ ]:
accuracy = accuracy_score(y_test, pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
predset = pd.DataFrame({'id': test['id'], 'Target': pred})
predset.head(3)

In [ ]:
def targetNames(col):
    if col == 0:
        return 'Dropout'
    if col == 1:
        return 'Enrolled'
    if col == 2:
        return 'Graduate'

prediction_df = pd.Series(pred).apply(targetNames)
preddf = pd.DataFrame({'id': test['id'], 'Target': prediction_df})
preddf.head(10)

In [ ]:
preddf.to_csv('sample_submission.csv', index=False)

In [ ]:
accuracy_score(sampledf,pred) 